<h1 style="text-align:center;"> BooN Benchmarking </h1>
Benchmark evaluating the computation time of the main BooN methods.
The parameters are adapted to provide complex formulas using ER topology and  simple formulas using SW topology.

In [1]:
import random

from boon import *
from datetime import datetime
from sympy import SOPform

from pulp import GUROBI_CMD
import os

## Stable states computation benchmark
Evaluate the time spent on computing the stable states.

In [ ]:
# PARAMETERS OF THE BENCHMARK
# MINIMAL NUMBER OF NODES
Nmin = 10
# MAXIMAL NUMBER OF NODES
Nmax=500
# NUMBER OF TRIALS
trials = 10
# STEPS
step = 10
# Open file for benchmark trace.
# trace_file = "bench-stable-state-ER.csv"
trace_file = "bench-stable-state-SW.csv"
ftrace =  open(trace_file, "w")
ftrace.write("# BENCHMARK: "+datetime.now().strftime("%d-%b-%y %H")+"\n")

for n in range(Nmin,Nmax+step,step):
    print("\r%3d : *"%n, end="")
    for trial in range(trials):
        # boon = BooN.random(n, p_link=0.25, p_pos=0.6, topology='Erdos-Reny')
        boon = BooN.random(n, p_link=1, p_pos=0.6, topology='Small-World')
    
        print("\r%3d : \\"%n, end="")
        
        # Total number of terms of the Boon
        totalterms = 0
        for formula in boon.desc.values():
            if not isinstance(formula, bool):
                for clause in formula.args:
                    totalterms+=max(len(clause.args),1)
                 
        print("\r%3d : |"%n, end="")
        # START TIME
        start = datetime.now()
        # PROCESS THE TASK 
        result = boon.stable_states
        # END TIMER
        end = datetime.now()
        duration = (end-start).total_seconds()
        # LINE CONTENT
        # size, trial number, number of stable states, number of terms, duration  
        ftrace.write("%d, %d, %d, %d,  %f\n"%(n,trial,len(result),totalterms, duration))
        print("\r%3d : ―"%n, end="")
        
    print("\r%3d : ."%n, end="")
    ftrace.flush()

print(">> END STABLE STATE BENCHMARK")
ftrace.close()

## Equilibria Computation Benchmark
Evaluate the time spent on computing the equilibria using the brute force method by generating the complete state graph.

In [ ]:
# PARAMETERS OF THE BENCHMARK
# MINIMAL NUMBER OF NODES
# Nmin = 1
Nmin = 1
# MAXIMAL NUMBER OF NODES
Nmax = 15
# NUMBER OF TRIALS
trials = 10
# STEPS
step = 1
# Open file for benchmark trace.
# trace_file = "bench-equilibria-ER.csv"
trace_file = "bench-equilibria-SW.csv"
ftrace =  open(trace_file, "a")  # The trace file cumulates the benchmarks.
ftrace.write("# BENCHMARK EQUILIBRIA: "+datetime.now().strftime("%d-%b-%y %H")+"\n")

for n in range(Nmin,Nmax+step,step):
    for trial in range(trials):
        # boon = BooN.random(n, p_link=0.5, p_pos=0.6, topology='Erdos-Reny')
        boon = BooN.random(n, p_link=0.1, p_pos=0.6, topology='Small-World')

         # Total number of terms of the Boon
        totalterms = 0
        for formula in boon.desc.values():
            if not isinstance(formula, bool):
                for clause in formula.args:
                    totalterms+=max(len(clause.args),1)

        print("\n%3d-%3d: ASYNCHONOUS  "%(n,trial))
        # START TIME
        start = datetime.now()
        # PROCESS THE TASK 
        model = boon.model(mode=asynchronous, trace=True)
        print("\rBenchmark >>  equilibria computation               ", end="")
        eqs_asynchronous = boon.equilibria(model=model, trace=True)
        print("\rBenchmark >> Completed                             ", end="")
        # END TIMER
        end = datetime.now()
        duration_asynchronous = (end-start).total_seconds()
        
        print("\n%3d-%3d: SYNCHRONOUS  "%(n,trial))
        start = datetime.now()
        # PROCESS THE TASK 
        model = boon.model(mode=synchronous, trace=True)
        print("\rBenchmark >>  equilibria computation               ", end="")
        eqs_synchronous = boon.equilibria(model=model, trace=True)
        print("\rBenchmark >> Completed                             ", end="")
        # END TIMER
        end = datetime.now()
        duration_synchronous = (end-start).total_seconds()
            # LINE CONTENT
        # size, trial number, number of equilibria for asynchronous mode, number of equilibria for synchronous mode, number of terms, asynchronous duration, synchronous duration  
        ftrace.write("%d, %d, %d, %d,  %d, %f, %f\n"%(n,trial,len(eqs_asynchronous),len(eqs_synchronous),totalterms, duration_asynchronous, duration_synchronous))
        ftrace.flush()
        print(">> ", end="*")
    ftrace.flush()

print(">> END EQUILIBRIA BENCHMARK")
ftrace.close()

 ## Controllability
In the controllability benchmark, we find the control to reach a Boolean profile a stable state for x0 x1 which is not contained in the stable states.
The possibility and the necessity are tested.

### Possibility

In [ ]:
# Benchmark Possibility
# PARAMETERS OF THE BENCHMARK
# MINIMAL NUMBER OF NODES
Nmin =80
# MAXIMAL NUMBER OF NODES
Nmax = 100
# NUMBER OF TRIALS
trials = 10
# STEPS
step = 10

# Open file for benchmark trace.
trace_file = "bench-possibility-ER.csv"  # Erdös Reny
# trace_file = "bench-possibility-SW.csv" # Small-World
ftrace = open(trace_file, "a")
ftrace.write("# BENCHMARK CONTROLLABILITY - POSSIBILITY: " + datetime.now().strftime("%d-%b-%y %H") + "\n")

x0, x1 = symbols('x0 x1')  # Define the variable.

for n in range(Nmin, Nmax + step, step):
    print("\r%3d    : *" % n, end="")
    for trial in range(trials):
        boon = BooN.random(n, p_link=0.5, p_pos=0.6, topology='Erdos-Reny')
        # boon = BooN.random(n, p_link=1, p_pos=0.6, topology='Small-World')
        print("\r%3d %3d: \\" % (n, trial), end="")

        # Total number of terms of the Boon
        totalterms = 0
        for formula in boon.desc.values():
            if not isinstance(formula, bool):
                for clause in formula.args:
                    totalterms += max(len(clause.args), 1)

        # Find a missing configuration in stable state
        stablestate = boon.stable_states
        eqpairs = {(eq[x0], eq[x1]) for eq in stablestate}
        freestates = {(True, True), (False, False), (True, False), (False, True)} - eqpairs

        # if freestates is empty meaning that all the Boolean configurations exist in the stable states, then skip the trial.
        if freestates:
            controlvalues = random.choice(list(freestates))
        else:
            continue

        # Define the query
        query = SOPform({x0, x1}, [dict(zip((x0, x1), controlvalues))])

        # Frozen variables excluding x0 x1
        frozenvars = boon.variables - {x0, x1}
        # Set control to boon
        boon.control(frozenvars, frozenvars)

        print("\r%3d %3d: |" % (n, trial), end="")
        # EVALUATE CONTROLLABILITY
        # POSSIBILITY
        start = datetime.now()
        # PROCESS THE TASK
        destiny = boon.possibly(query)
        core = boon.destify(destiny, max_solutions=100, trace=True)
        actions_possibly = core2actions(core)
        # END TIMER
        end = datetime.now()
        duration_possibly = (end - start).total_seconds()

        print("\r%3d %3d: ―" % (n, trial), end="")

        # LINE CONTENT
        # size, trial number, number of solutions for possibility, number of terms, possibility duration
        ftrace.write("%d, %d, %d, %d, %f\n" % (n, trial, len(actions_possibly), totalterms, duration_possibly))
        ftrace.flush()
        print("\r%3d-%3d: /" % (n, trial), end="")

    print("\r%3d    : ." % n, end="")
    ftrace.flush()

print(">> END CONTROLLABILITY POSSIBILITY BENCHMARK")
ftrace.close()

BooN PI >> Solve.                                  
 80   1: |